In [2]:
!pip install scikit-learn==0.24.1  aif360==0.3.0  tensorflow==2.4.0 nodejs==0.1.1 ipywidgets==7.6.3 lime==0.2.0.1 import_ipynb==0.1.3 #aix360==0.2.1 

In [ ]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [4]:
import tensorflow as tf
from tensorflow import keras
import import_ipynb
from codait_utils import *

importing Jupyter notebook from codait_utils.ipynb


In [5]:
import os
target_column = os.environ.get('target_column','target')
protected_column = os.environ.get('protected_column')
prediction_column = os.environ.get('prediction_column','prediction')
unpriviledged_group_key = os.environ.get('unpriviledged_group_key')
priviledged_group_key = os.environ.get('priviledged_group_key')
metadata = os.environ.get('metadata','metadata.csv')

In [6]:
def is_local():
    import socket
    return '172.17.0.2' == socket.gethostbyname(socket.gethostname())

In [7]:
if is_local():
    target_column = 'finding'
    protected_column = 'sex'
    unpriviledged_group_key = 'F'
    priviledged_group_key = 'M'

In [ ]:
unzip('.','model.zip')
unzip('.','data.zip')

In [8]:
import pandas as pd
df = pd.read_csv(metadata)

In [9]:
df

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29,prediction
0,2,0.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,No Finding
1,2,3.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,No Finding
2,2,5.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,No Finding
3,2,6.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,No Finding
4,4,0.0,F,52.0,Pneumonia_Viral_COVID-19,Y,NaN,N,N,N,...,"Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN,No Finding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,479,0.0,F,40.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,United Kingdom,images,072ecaf8c60a81980abb57150a8016_jumbo-9.jpeg,NaN,https://radiopaedia.org/cases/multifocal-round...,CC BY-NC-SA,Asthmatic. Shortness of breath and wheeze. Rou...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN,No Finding
925,479,70.0,F,40.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,United Kingdom,images,ff33c406392b968d483174c97eb857_jumbo-9.jpeg,NaN,https://radiopaedia.org/cases/multifocal-round...,CC BY-NC-SA,Asthmatic. Shortness of breath and wheeze. The...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN,No Finding
926,480,NaN,M,26.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,images,000001-266.jpg,NaN,https://www.eurorad.org/case/947,CC BY-NC-SA 4.0,fire-eater accidentally ingested a paraffin mi...,NaN,NaN,No Finding
927,481,NaN,M,50.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,images,000001-272.jpg,NaN,https://www.eurorad.org/case/934,CC BY-NC-SA 4.0,"The patient, a heavy smoker, was referred to t...",NaN,NaN,No Finding


In [10]:
df["protected_column_index"] = df[protected_column].apply(lambda x: list(df[protected_column].unique()).index(x))
df["missclassified"] = df.apply(lambda d: 1 if d[target_column]!=d[prediction_column] else 0, axis=1)

In [11]:
from aif360.datasets import BinaryLabelDataset

unprivileged_groups = [{'protected_column_index': df.loc[df[protected_column] == unpriviledged_group_key].iloc[0]['protected_column_index']}]
privileged_groups = [{'protected_column_index': df.loc[df[protected_column] == priviledged_group_key].iloc[0]['protected_column_index']}]
                     
                   
favorable_label = 0
unfavorable_label = 1 #missclassified == True

df_for_aif360 = df[["protected_column_index", "missclassified"]]



df_for_aif360

,protected_column_index,missclassified
0,0,1
1,0,1
2,0,1
3,0,1
4,1,1
...,...,...
924,1,1
925,1,1
926,0,1
927,0,1


In [12]:
aif360_dataset = BinaryLabelDataset(
    favorable_label=favorable_label,
    unfavorable_label=unfavorable_label,
    df=df_for_aif360,
    label_names=['missclassified'],
    protected_attribute_names=['protected_column_index'],
    unprivileged_protected_attributes=unprivileged_groups)

In [13]:
from aif360.metrics import BinaryLabelDatasetMetric

metric_orig_train = BinaryLabelDatasetMetric(aif360_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

In [14]:
metric_orig_train.statistical_parity_difference()

0.009556450150509554

In [15]:
metric_orig_train.smoothed_empirical_differential_fairness()

0.523248143764548